In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [79]:
def scrape_tracking(url, years, metric):
    driver = webdriver.Firefox(executable_path=r"C:\Users\spens\Basketball\geckodriver.exe")
    col_names = []
    row_entries = []
    for year in years:
        print(year)
       # print(url.format(metric, year))
        driver.get(url.format(metric, year))
        time.sleep(10)
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,"onetrust-accept-btn-handler")))
            driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
        except:
            print()
        time.sleep(10)
       # path = "/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/div/table/thead/tr/th[{}]"
       # path = path.format(n)
       # driver.find_element_by_xpath(path).click()
        
        res = Select(driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))
        res.select_by_visible_text('All')
       
        th_elements = driver.find_elements(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/div/table/thead/tr/th')
        #print(type(th_elements))
        #print(len(th_elements))
        time.sleep(2)
        cols = []
        for elt in th_elements:
            cols.append(str(elt.text))
        tr_elements = driver.find_elements(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/div/table/tbody/tr')
        #print(type(tr_elements))
        #print(len(tr_elements))
        #time.sleep(0.5)
        rows = []
        for elt in tr_elements:
            row = str(elt.text)
            row = row.split('\n')
            stats = row[1].split(' ')
            lst = [row[0]] + stats
            rows.append(lst)
        col_names.append(cols)
        row_entries.append(rows)
    driver.quit()
    return col_names, row_entries

In [80]:
def clean_tracking(col_names, row_entries, years):
    #print(col_names)
    every_szn = []
    for i in range(len(years)):
        temp_col = col_names[i]
        #print(len(temp_col))
        temp_col = [x for x in temp_col if x != ""]
        #print(len(temp_col))
        temp_row = row_entries[i]
        year = years[i]
        temp_col.append('Year')
        df = pd.DataFrame(columns=temp_col)
        #print(df)
        for r in temp_row:
            #print(r)
            r.append(year)
            #df.loc[-1] = r
            #df.index = df.index + 1
            #df = df.sort_index()
            df = df.append(pd.Series(r, index=df.columns[:len(r)]), ignore_index=True)
        #print(df)
        every_szn.append(df)
    master = pd.concat(every_szn)
    return master

In [81]:
track_pages = ["defensive-impact","drives","catch-shoot","passing","touches","pullup","rebounding","shooting-efficiency","speed-distance","elbow-touch","tracking-post-ups","paint-touch"]
url = 'https://www.nba.com/stats/players/{}/?Season={}&SeasonType=Playoffs'
years = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20']
#years= ['2013-14']

In [82]:
df_lst = []
for metric in track_pages[11:12]:
    print(metric)
    cols, rows = scrape_tracking(url, years, metric)
    df = clean_tracking(cols, rows, years)
    metric_col = [str(metric) for i in range(len(df))]
    df['Metric'] = metric_col
    df_lst.append(df)
    print()

paint-touch


C:\Users\spens\AppData\Local\Temp/ipykernel_21020/2306852913.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r"C:\Users\spens\Basketball\geckodriver.exe")


2013-14
2014-15

2015-16

2016-17

2017-18

2018-19

2019-20




In [83]:
for df in df_lst:
    #print(df.dtypes)
    cols = df.columns.drop(['PLAYER', 'TEAM', 'Year', 'Metric'])
    df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")
    df = df.loc[df['GP'] > 50]

In [84]:
df_lst[0].to_csv(r"C:\Users\spens\Basketball\NBA Tracking\Tracking Data\track_paint-touch_playoff_szn.csv")